In [1]:
!pip install selenium


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install lxml


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
!pip install html5lib


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import requests
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
import time
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
from selenium.webdriver.support.ui import Select

In [4]:
# flavorDB 사이트에서 크롤링할 재료의 카테고리 목록 초기화
categories = [
    "Additive","Animal Product","Bakery", "Berry", "Beverage Alcoholic", "Beverage Caffeinated", "Beverage", 
    "Cabbage", "Cereal", "Dairy", "Dish", "Essential Oil", "Fish", "Flower", "Fruit Citrus", 
    "Fruit","Fruit Essence","Fungus", "Gourd", "Herb", "Legume", "Maize", "Meat", "Nut", "Plant Derivative", "Plant", 
    "Seafood", "Seed", "Spice", "Vegetable Fruit", "Vegetable Root", "Vegetable Stem", "Vegetable"
]

all_entities = []

# 크롤링을 위한 chromedriver와 url 세팅 
s = Service("C:/Users/rla00/2023ksc_1crawling/chromedriver.exe")
driver = webdriver.Chrome(service=s)
url = "https://cosylab.iiitd.edu.in/flavordb/search"

for category in categories:
    driver.get(url)
    
    # 크롤링 과부하 방지를 위한 time.sleep 
    time.sleep(3)

    # 크롤링의 바탕이 되는 url에서 'Entities/Ingredients' 탭을 클릭 
    ingredients_tab = driver.find_element(By.XPATH, "//a[@href='#ingredients']")
    ingredients_tab.click()

    # 탭 클릭 후 로딩을 위한 time.sleep
    time.sleep(6)
    
    # 각 카테고리별 해당 재료를 100개씩 크롤링 하기 위해 dropdown 메뉴를 찾고100 자동 입력
    dropdown_element = driver.find_element(By.NAME, "log_table_entities_length")
    dropdown = Select(dropdown_element)
    dropdown.select_by_value("100")

    # 위의 조건대로 각 카테고리 별 재료 검색 : 검색 자동화
    category_input = driver.find_element(By.ID, "category")
    category_input.clear() # 다음 카테고리 검색을 위한 검색창 초기화 
    category_input.send_keys(category) # 검색 필드에 초기화한 카테고리 목록들을 순회하며 입력
    category_input.send_keys(Keys.RETURN) # 엔터 키를 눌러서 검색을 실행
    
    time.sleep(3)

    wait = WebDriverWait(driver, 10)

    while True:
        try:
            # 특정 div 태그 안에 row가 나타날 때까지 대기
            wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='col-sm-12']//table[@id='log_table_entities']//tbody//tr//td//a[@class='text-capitalize']")))

            # Xpath를 통해 현재 페이지의 elements를 찾음
            entity_elements_xpath = "//div[@class='col-sm-12']//table[@id='log_table_entities']//tbody//tr//td//a[@class='text-capitalize']"
            entity_elements = driver.find_elements(By.XPATH, entity_elements_xpath)
            
            # 각 엔터티에 대해 이름과 링크를 수집하고 all_entities 리스트에 추가
            for entity_element in entity_elements:
                entity_name = entity_element.text
                entity_link = entity_element.get_attribute('href')
                all_entities.append({"category": category, "name": entity_name, "link": entity_link})

            # "다음" 버튼이 있는지 확인
            next_button_li = driver.find_element(By.XPATH, "//div[@id='log_table_entities_paginate']//li[@class='paginate_button next']")
            
            # "다음" 버튼이 활성화되어 있는지 확인
            if "disabled" not in next_button_li.get_attribute("class"):
                next_button_a = next_button_li.find_element(By.XPATH, "./a")
                next_button_a.click()
                
                # '다음' 버튼이 stale 상태가 될 때까지 대기하며 페이지 re-load
                wait.until(EC.staleness_of(next_button_a))
                
                # 새 페이지가 콘텐츠를 로드 할 때까지 대기
                wait.until(EC.presence_of_element_located((By.XPATH, entity_elements_xpath)))
            else:
                # "다음" 버튼이 비활성화되어 있다면 무한 루프 종료
                break

        except TimeoutException:
            print(f"Timeout occurred for {category}, potentially last page reached or there was a delay in loading the page.")
            break
        except Exception as e:
            print(f"Unexpected error for {category}: {e}")
            break

# 크롤링한 데이터를 CSV 파일로 저장
import csv

filename = "entities.csv"
with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["category", "name", "link"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for entity in all_entities:
        writer.writerow(entity)

print(f"Data saved to {filename}")


Unexpected error for Additive: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@id='log_table_entities_paginate']//li[@class='paginate_button next']"}
  (Session info: chrome=119.0.6045.200)
Stacktrace:
	GetHandleVerifier [0x00007FF6552682B2+55298]
	(No symbol) [0x00007FF6551D5E02]
	(No symbol) [0x00007FF6550905AB]
	(No symbol) [0x00007FF6550D175C]
	(No symbol) [0x00007FF6550D18DC]
	(No symbol) [0x00007FF65510CBC7]
	(No symbol) [0x00007FF6550F20EF]
	(No symbol) [0x00007FF65510AAA4]
	(No symbol) [0x00007FF6550F1E83]
	(No symbol) [0x00007FF6550C670A]
	(No symbol) [0x00007FF6550C7964]
	GetHandleVerifier [0x00007FF6555E0AAB+3694587]
	GetHandleVerifier [0x00007FF65563728E+4048862]
	GetHandleVerifier [0x00007FF65562F173+4015811]
	GetHandleVerifier [0x00007FF6553047D6+695590]
	(No symbol) [0x00007FF6551E0CE8]
	(No symbol) [0x00007FF6551DCF34]
	(No symbol) [0x00007FF6551DD062]
	(No symbol) [0x00007FF6551CD3A3]
	BaseThreadInitThunk [0x00007FFBA563257D+29]

PermissionError: [Errno 13] Permission denied: 'entities.csv'

In [5]:
import pandas as pd 

In [6]:
data = pd.read_csv("entities.csv")

In [7]:
# link column으로 중복된 크롤링 데이터 제거
data_unique = data.drop_duplicates(subset='link', keep='first')

# 중복 제거된 전처리 csv 파일 저장
data_unique.to_csv("entities_cleaned.csv", index=False)

print(f"Data saved to entities_cleaned.csv with {len(data_unique)} unique entities.")

Data saved to entities_cleaned.csv with 936 unique entities.


In [14]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# 전처리된 csv 파일 load
data = pd.read_csv("entities_cleaned.csv")

# 각 재료에 대한 link column에는 해당 재료의 분자 종류에 따른 flavor profile 데이터(분자 - 맛)가 존재
# 이 데이터를 추출할 함수에 대한 정의
def get_flavor_profiles(url):
    try:
        response = requests.get(url) # url로부터 응답을 받음
        response.raise_for_status()  # 응답이 200 OK가 아닌 경우 에러 발생

        # BeautifulSoup을 사용하여 HTML 파싱
        #soup = BeautifulSoup(response.content, "lxml")
        soup = BeautifulSoup(response.content, "html.parser")

        body = soup.find("tbody")
        profiles = body.find_all("td", class_=False)

        flavors = []
        for i in range(1, len(profiles), 3):
            flavors.append(profiles[i].text.strip())

        return flavors

    except requests.HTTPError as http_err:
        print(f"HTTP error occurred while accessing {url}: {http_err}")
    except Exception as err:
        print(f"Other error occurred while accessing {url}: {err}")

    return None

# 각 행의 'link' column에 함수를 적용하고 결과를 새로운 'flavor profiles' column을 추가하여 저장
data['flavor profiles'] = data['link'].apply(get_flavor_profiles)

# 각 재료 - 맛에 대한 csv 파일 저장
data.to_csv("entities_with_flavors.csv", index=False)

print("Finished scraping flavor profiles!")


100%|████████████████████████████████████████████████████████████████████████████████| 936/936 [18:16<00:00,  1.17s/it]


Finished scraping flavor profiles!


In [15]:
data = pd.read_csv("entities_with_flavors.csv")

In [16]:
data["flavor profiles"]

0      ['sweet, new mown hay, green, tonka, bitter', ...
1      ['mild, sour, bitter', 'grassy, very mild', 's...
2      ['bitter', 'grassy, very mild', 'bland', 'sour...
3      ['bitter', 'mild, sour, bitter', 'grassy, very...
4      ['mild, sour, bitter', 'grassy, very mild', 's...
                             ...                        
931                      ['bitter', 'grassy, very mild']
932    ['sweet, new mown hay, green, tonka, bitter', ...
933    ['sweet, new mown hay, green, tonka, bitter', ...
934    ['sweet, new mown hay, green, tonka, bitter', ...
935    ['sweet, new mown hay, green, tonka, bitter', ...
Name: flavor profiles, Length: 936, dtype: object

In [17]:
import pandas as pd

data = pd.read_csv("entities_with_flavors.csv")

# 문자열 표현을 리스트로 변환하는 과정
import ast  

def clean_flavor(flavor_str):
    # 문자열 표현을 실제 리스트로 변환
    flavor_list = ast.literal_eval(flavor_str)
    
    cleaned_flavors = [flavor.strip() for flavors in flavor_list for flavor in flavors.split(',')]
    
    # 쉼표로 구분된 문자열로 다시 변환
    return ', '.join(cleaned_flavors)



# 'flavor profiles' column에 위에서 정의된 함수를 적용하여 데이터 정리
data['flavor profiles'] = data['flavor profiles'].apply(clean_flavor)

# CSV 파일로 저장
data.to_csv("entities_with_flavors_cleaned.csv", index=False)



In [18]:
import pandas as pd

# 'Not Available' 제거 전처리 과정
data = pd.read_csv("entities_with_flavors_cleaned.csv")

data['flavor profiles'] = data['flavor profiles'].apply(lambda x: ', '.join([flavor for flavor in str(x).split(', ') if flavor != 'Not Available']))

data.to_csv("entities_with_flavors_cleaned.csv", index=False)

print("Removed 'Not Available' from flavor profiles.")


Removed 'Not Available' from flavor profiles.


In [19]:
import pandas as pd

data = pd.read_csv("entities_with_flavors_cleaned.csv")

# set으로 만들어서 unique한 재료에 대한 최종 파일을 얻고자 함 
unique_flavors = set(flavor.strip() for flavors in data['flavor profiles'].dropna() for flavor in flavors.split(','))

# 알파벳 순 정렬
sorted_flavors = sorted(unique_flavors)

# .txt file로 최종 저장
with open("unique_flavors.txt", "w") as f:
    for flavor in sorted_flavors:
        f.write(f"{flavor}\n")

